# Evaluation


In [16]:
with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [17]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


In [18]:
import torch

device = "cpu"
if torch.cuda.is_available():
    print("Cuda available")
    device = torch.device('cuda')

Cuda available


### Preparing the Data for Evaluation

In [19]:
import os 
eval_path = r"Evaluation data\Solved PDFs"
solved_pdfs_path = os.listdir(eval_path)
solved_pdfs_path_pdf_files = [os.path.join(eval_path, pdf) for pdf in solved_pdfs_path if pdf.endswith("pdf")]
solved_pdfs_path_txt_files = [os.path.join(eval_path, txt) for txt in solved_pdfs_path if txt.endswith("txt")]

In [20]:
list_tuple_data = []
i = 0
while i < len(solved_pdfs_path_pdf_files):
    tup = (solved_pdfs_path_pdf_files[i], solved_pdfs_path_txt_files[i])
    list_tuple_data.append(tup)
    i += 1

In [21]:
list_tuple_data

[('Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.pdf',
  'Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.txt'),
 ('Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.pdf',
  'Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.txt'),
 ('Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.pdf',
  'Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.txt'),
 ('Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.pdf',
  'Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.txt'),
 ('Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.pdf',
  'Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.txt')]

### Loading Prompts

In [22]:
import json

with open("prompts_preprocessing.json", "r") as f:
    prompts_for_processing = json.load(f)

table_process_prompt = prompts_for_processing["augment_table_prompt"]
summary_prompt = prompts_for_processing["summary_prompt"]

In [23]:
with open("french_prompts.json", "r") as f:
    prompts_query = json.load(f)

## Loading markdown

-2 hecho
-3 broken

In [24]:
pdf_path = list_tuple_data[-5][0]
markdown_path = list_tuple_data[-5][1]

In [25]:
pdf_path

'Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.pdf'

In [26]:
with open(markdown_path, "r") as f:
    markdown = f.read()

In [27]:
markdown

'# Accusé de réception en préfecture\n\n067-200067783-20231214-231214D13-DE\n\nDate de télétransmission : 15/12/2023\n\nDate de réception préfecture : 15/12/2023\n\n# Ville de Bouxwiller\n\npetersbach\n\net la communauté agrocléel\n\n# CONVENTION DE PARTENARIAT\n\n# DANS LE CADRE DU CONTRAT DE TERRITOIRE OUEST ALSACE 2022-2025\n\nPour agir sur les projets de structures périscolaires de Bouxwiller et Petersbach par la Communauté de Communes de Hanau La Petite Pierre\n\n# Entre\n\nLa Collectivité européenne d\'Alsace, représentée par son Président, Monsieur Frédéric BIERRY, dûment habilité par délibération no CD-2023-4-10-1 du Conseil de la Collectivité européenne d\'Alsace du 13 novembre 2023,\n\nCi-après dénommée « la Collectivité européenne d\'Alsace » ou « la CeA »,\n\nET\n\nLa Communauté de Communes de Hanau La Petite Pierre, représentée par son Président, Monsieur Patrick MICHEL, dûment habilité par délibération no du Conseil Communautaire du,\n\nCi-après dénommée « la Communauté d

## Processing

In [28]:
from council_rag.preprocessing import preprocess_markdown_text
from council_rag.data_transformations import process_tables, summarize_clusters
from council_rag.preprocessing.preprocessing import unload_cuda

import time

unload_cuda()
start = time.time()

paragraphs_list, clusters_dict, model = preprocess_markdown_text(markdown,
                                                        model_id ="Jaume/gemma-2b-embeddings", 
                                                        spacy_model="fr_core_news_sm", 
                                                        n_sents_per_para=8,
                                                        device=device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.39it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 9.34 GiB is allocated by PyTorch, and 13.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
unload_cuda()
processed_tables = process_tables(pdf_path,
                                eval_path, # required to know where to save the pdf images
                                table_process_prompt, 
                                groq_token)

In [16]:
unload_cuda()
clusters_dict = summarize_clusters(clusters_dict, 
                                    summary_prompt, 
                                    groq_token, 
                                    model="gemma2-9b-it", 
                                    token_limit=14000, 
                                    sleep_time=60)

end = time.time()
print((end - start)/60)

3.8146627227465313


In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from council_rag.rag import prepare_data_for_rag

unload_cuda()
clusters_list, paragraphs_list, all_docs = prepare_data_for_rag(clusters_dict, 
                                                                paragraphs_list, 
                                                                processed_tables, 
                                                                splitter=RecursiveCharacterTextSplitter,
                                                                chunk_size=450,
                                                                chunk_overlap=35,
                                                                length_function=len,
                                                                is_separator_regex=False)

Cuda available


In [18]:
from council_rag.rag import shorten_summary_docs

unload_cuda()
all_docs = shorten_summary_docs(all_docs, groq_token)

In [20]:
unload_cuda()


616

In [26]:
from council_rag.rag import set_up_rag_index 

unload_cuda()
vector_store, index, emd_dims = set_up_rag_index(model)

In [29]:
from council_rag.rag import populate_vector_store

unload_cuda()
vector_store, all_embeddings = populate_vector_store(vector_store, all_docs, model)

In [ ]:
from council_rag.rag import raptor_query_all_prompts

unload_cuda()
list_aug_queries = raptor_query_all_prompts(vector_store, prompts_query) # queries in ["formated_augmented_query"]

In [64]:
from council_rag.rag import return_final_responses

final_responses = return_final_responses(list_aug_queries, groq_token)

In [73]:
path_response = pdf_path.replace(".pdf", ".json") # saved in the same folder as the pdf
with open(path_response, "w") as f:
    json.dump(final_responses, f)

In [65]:
final_responses

[{'prompt': 'is_geo',
  'formated_augmented_query': "Vous êtes une personne qui répond aux questions de l'IA. Un utilisateur va vous transmettre un texte de description de projet en français. Votre tâche consiste à déterminer si le texte concerne un projet géothermique ou s'il n'a aucun rapport avec un projet géothermique. Prenez en compte les mentions suivantes : - Installations d'énergie géothermique - Etudes ou enquêtes géothermiques - Systèmes de pompes à chaleur utilisant l'énergie géothermique - Chauffage/refroidissement par le sol - Forage pour les ressources géothermiques - Mots clés comme « géothermie », « géothermique » - Nature du document Assurez-vous de générer une série d'étapes logiques et de raisonner sur le problème avant de générer une solution.\n\nVoici quelques faits pertinents pour vous aider à répondre et una description du qui peut s'avérer utile. \n    Si la description du n'est pas utile, ignorez-le.\nun maximum à leurs demandes ce qui justifie le coût du proje